In [21]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from datacube import helpers

In [23]:
help(helpers.write_geotiff)

Help on function write_geotiff in module datacube.helpers:

write_geotiff(filename, dataset, profile_override=None, time_index=None)
    Write an ODC style xarray.Dataset to a GeoTIFF file.
    
    :param filename: Output filename
    :param dataset: xarray dataset containing one or more bands to write to a file.
    :param profile_override: option dict, overrides rasterio file creation options.
    :param time_index: DEPRECATED



In [24]:
#load modules
import datacube
from datacube.storage import masking
from datacube import Datacube
from datetime import datetime
from skimage import exposure

import rasterio
from datacube_stats.statistics import GeoMedian
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import geopandas as gpd
from sklearn.ensemble import ExtraTreesClassifier
import datacube_stats
import os
# Replace '156' and 'ck9738' with the path to your own home directory on the VDI
s2aws = Datacube(config='/g/data/u46/users/ck9738/datacube-s2ard.conf')

In [ ]:
query = {
        'lat': (-35.25, -35.35),
        'lon': (149.05, 149.17),
        'output_crs': 'EPSG:3577',
        'resolution': (-10, 10),
        'time':('2017-01-01', '2017-02-15')
        }

Canberra = s2aws.load(product='s2a_ard_granule', group_by='solar_day', **query)

In [ ]:
Canberra

In [ ]:
slope= (Canberra.nbart_coastal_aerosol.isel(time=1))

slope.shape

In [ ]:
#view your awsone classification output
fig, ax = plt.subplots(figsize = (12,12))
cax = ax.imshow(slope)

plt.show()

In [ ]:
    'nbart_coastal_aerosol','nbart_blue','nbart_green','nbart_red','nbart_red_edge_1','nbart_red_edge_2','nbart_red_edge_3','nbart_nir_1','nbart_nir_2', 'nbart_swir_2', 'nbart_swir_3' 

In [ ]:
#define nessisary gotiff handeling fuctions


import xarray, rasterio, numpy as np
def numpy_to_xarray(array, geobox, name=None):
    """Utility to convert ndarray to DataArray, using a datacube.model.GeoBox"""
    coords=[xarray.Coordinate(x, geobox.coords[x].values, attrs=dict(units=geobox.coords[x].units)) 
            for x in geobox.dims]
    return xarray.DataArray(array, coords=coords, attrs=dict(crs=geobox.crs), name=name)

def geopandas_to_xarray(table, geobox, name=None):
    """Rasterise (with reprojection)"""
    array = rasterio.features.rasterize(shapes=table.to_crs(geobox.crs._crs.ExportToProj4()).geometry,
                                        out_shape=(geobox.height, geobox.width),
                                        transform=geobox.affine)
    return numpy_to_xarray(array, geobox, name)

def rasterfile_to_xarray(file, geobox, name=None, nodata=True, num_bands=(1), X_band= 1):
    """Blit like
    """
    with rasterio.open(file) as src:

        #|assert src.indexes == num_bands # assume single band
        band = rasterio.band(src, X_band) # do not attempt to read entire extent into memory
        array = np.empty((geobox.height, geobox.width), dtype=band.dtype)
        rasterio.warp.reproject(source=band,
                                destination=array,
                                dst_crs=geobox.crs.crs_str,
                                dst_transform=geobox.affine,
                                dst_nodata=nodata)
    return numpy_to_xarray(array, geobox, name)
    
def array_topanda_s(array):
    a,b = array.shape
    length_be = int(a)+int(b)
    try:
        flat = array.stack(z=('y','x'))
    except:
        flat = array.stack(z=('longitude','latitude'))
    #ones_array = np.ones_like(flat)
    #ones_array = np.where(flat)
    panda_series = pd.Series(flat)
    return(panda_series)

In [ ]:
def make_coords(filename):
    shapefile_loc='/g/data/u46/users/ck9738/Datasets/ML_for_smad_green/shapefiles_for_train/GM_train_multi_bioregion/'+filename

    #import project area shapefiles
    project_area = gpd.read_file(shapefile_loc)

    #convert the shapefile to GDA94 lat-long coords so we can query dc_load using lat long
    project_area['geometry'] = project_area['geometry'].to_crs(epsg=4283)

    #find the bounding box that contains all the queried projects

    coords = project_area.total_bounds
    #print(coords)
    xmin, ymax, xmax, ymin =coords
    return(xmin, ymax, xmax, ymin)

def load_data(xmin, ymax, xmax, ymin, bands):#load sentinal data
    sensors= ['s2a_ard_granule'] #pick the sentinal satelites you want

    bands_of_int = bands
    
    query = {
            'lat': (ymin, ymax),
            'lon': (xmin, xmax),
            'output_crs': 'EPSG:3577',
            'resolution': (-10, 10),
            'time':('2015-01-01', '2018-03-30')
            }
      
# use s2b_ard_granule 	 for S2B
    data_sent = s2aws.load(product='s2a_ard_granule', measurements=bands, group_by='solar_day', **query)
    pixle_q = s2aws.load(product='s2a_ard_granule', measurements=('fmask','nbart_red'), group_by='solar_day', **query)
    clear = np.logical_and(pixle_q.fmask != 2, pixle_q.fmask != 3)
    clear_data = data_sent.where(clear)
    #print(data_sent)
    #cast the data to float numbers so geomedian calculation will work
    data_sent_flt = clear_data.astype(float,casting='safe')

    #print(pre_canned_gm)
#create geomedian for data
    Sentinal_gm=GeoMedian().compute(data_sent_flt)
    #data_sent= data_sent.isel(time=0)
    return(Sentinal_gm)
#data_sent = data_sent.where(clear_pixels)

In [ ]:
master_panda = pd.DataFrame()
#filelist1=['ncas_base_1.shp','ncas_base_2.shp','ncas_base_3.shp','ncas_base_4.shp','ncas_base_5.shp','ncas_base_6.shp','ncas_base_7.shp',]

In [ ]:
#filelist = ['canberra_minier.shp']
filelist1 = ['ausalps_1.shp']#,'ausalps_3.shp','ausalps_4.shp','ausalps_5.shp']#,'ausalps_1.shp'
#filelist2 = ['ausalps_2.shp']#,'ausalps_1.shp'
master_panda = pd.DataFrame()

In [ ]:
#sent_data_array = xr.concat([group_sent_bands[1],group_sent_bands[2],group_sent_bands[3],
#                              group_sent_bands[4],group_sent_bands[5],group_sent_bands[6],
#                              group_sent_bands[7],group_sent_bands[8],group_sent_bands[9],
#                              group_sent_bands[10],group_sent_bands[11]], 'band')


In [ ]:


for files in filelist1:
    bands = ['nbart_coastal_aerosol','nbart_blue','nbart_green','nbart_red','nbart_red_edge_1','nbart_red_edge_2','nbart_red_edge_3','nbart_nir_1','nbart_nir_2', 'nbart_swir_2', 'nbart_swir_3']
    
    
    xmin, ymax, xmax, ymin = make_coords(files)
    sent_data = load_data(xmin, ymax, xmax, ymin, bands)
    sent_data
    plot_sent = sent_data[['nbart_red','nbart_green','nbart_blue']].to_array()
    plot_sent.plot.imshow(robust=True)
    plt.show()
    #print(sent_data)
    #use sent geobox to load up ncas product and create class key
    #class_series = rasterfile_to_xarray('/g/data/u46/users/ck9738/Datasets/ML_for_smad_green/shapefiles_train_4class/c3classez_two.tif',
                               #sent_data.geobox, num_bands=(1), X_band= 1)
    ncas = rasterfile_to_xarray("/g/data/u46/users/fxy120/australian_woody/mosaics/lztmre_aus_y20002011_dm7a2_d20050630.tif",
                               sent_data.geobox, num_bands=(1), X_band= 1)
    
    ncas_class = np.logical_and(ncas>124, ncas<200)
    

    
    #ncas_class = np.logical_and(ncas>124, ncas<200)

    class_series = ncas_class*1
 
    
    little_panda = pd.DataFrame(array_topanda_s(class_series), columns=['class'])
    
    #sent_flat = sent_data.stack(z=('y','x'))
    
    #calculate all of the indexies
    #NDVI
#     ndvi = (sent_data['t_nir1']-sent_data['t_red'])/(sent_data['t_nir1']+sent_data['t_red'])
#     little_panda['ndvi']=array_topanda_s(ndvi)
    
#     #NDWI
#     ndwi = (sent_data['t_nir1']-sent_data['t_swir1'])/(sent_data['t_nir1']+sent_data['t_swir1'])
#     little_panda['ndwi']=array_topanda_s(ndwi)
#     # EVI
#     evi = 2.5*(sent_data['t_nir2']-sent_data['t_rededge1'])/(sent_data['t_nir2']+6*sent_data['t_rededge1']-7.5*sent_data['t_aerosol'])+1
#     little_panda['evi']=array_topanda_s(evi)
    
#     #CCCI
#     ccci = ((sent_data['t_nir2']-sent_data['t_rededge1'])/(sent_data['t_nir2']+sent_data['t_rededge1']))/((sent_data['t_nir2']-sent_data['t_red'])/(sent_data['t_nir2']+sent_data['t_red']))
#     little_panda['ccci']=array_topanda_s(ccci)
    
    for band in bands:
        little_panda[band] = array_topanda_s(sent_data[band])
    
    #load smad_green product
#     band_list = [1,2,3,4,5,6,7,8,9,10]
#     for band in band_list:

#         data = rasterfile_to_xarray("/g/data/r78/ck9738/smad_green_files/output/canberra_gm_2016_copy.tif",
#                                sent_data.geobox, num_bands=(1), X_band= band)
#         little_panda[str(band)] = array_topanda_s(data)

            
    
    master_panda = master_panda.append(little_panda,ignore_index=True)
master_panda

for files in filelist2:
    bands = ['t_aerosol','t_blue', 't_green', 't_red', 't_rededge1', 
             't_rededge2', 't_rededge3', 't_nir1', 't_nir2', 't_swir1', 't_swir2']
    
    
    xmin, ymax, xmax, ymin = make_coords(files)
    sent_data = load_data(xmin, ymax, xmax, ymin, bands)
    sent_data
    plot_sent = sent_data[['t_red','t_green','t_blue']].to_array()
    plot_sent.plot.imshow(robust=True)
    plt.show()
    #print(sent_data)
    #use sent geobox to load up ncas product and create class key
    #class_series = rasterfile_to_xarray('/g/data/u46/users/ck9738/Datasets/ML_for_smad_green/shapefiles_train_4class/c3classez_two.tif',
                               #sent_data.geobox, num_bands=(1), X_band= 1)
    ncas = rasterfile_to_xarray("/g/data/u46/users/fxy120/australian_woody/mosaics/lztmre_aus_y20002011_dm7a2_d20050630.tif",
                               sent_data.geobox, num_bands=(1), X_band= 1)
    
    ncas_class = np.logical_and(ncas>134, ncas<200)
    

    
    #ncas_class = np.logical_and(ncas>124, ncas<200)

    class_series = ncas_class*1
 
    
    little_panda = pd.DataFrame(array_topanda_s(class_series), columns=['class'])
    
    #sent_flat = sent_data.stack(z=('y','x'))
    
    #calculate all of the indexies
    #NDVI
    ndvi = (sent_data['t_nir1']-sent_data['t_red'])/(sent_data['t_nir1']+sent_data['t_red'])
    little_panda['ndvi']=array_topanda_s(ndvi)
    
    #NDWI
    ndwi = (sent_data['t_nir1']-sent_data['t_swir1'])/(sent_data['t_nir1']+sent_data['t_swir1'])
    little_panda['ndwi']=array_topanda_s(ndwi)
    # EVI
    evi = 2.5*(sent_data['t_nir2']-sent_data['t_rededge1'])/(sent_data['t_nir2']+6*sent_data['t_rededge1']-7.5*sent_data['t_aerosol'])+1
    little_panda['evi']=array_topanda_s(evi)
    
    #CCCI
    ccci = ((sent_data['t_nir2']-sent_data['t_rededge1'])/(sent_data['t_nir2']+sent_data['t_rededge1']))/((sent_data['t_nir2']-sent_data['t_red'])/(sent_data['t_nir2']+sent_data['t_red']))
    little_panda['ccci']=array_topanda_s(ccci)
    
    for band in bands:
        little_panda[band] = array_topanda_s(sent_data[band])
    
    #load smad_green product
#     band_list = [1,2,3,4,5,6,7,8,9,10]
#     for band in band_list:

#         data = rasterfile_to_xarray("/g/data/r78/ck9738/smad_green_files/output/canberra_gm_2016_copy.tif",
#                                sent_data.geobox, num_bands=(1), X_band= band)
#         little_panda[str(band)] = array_topanda_s(data)

            
    
    master_panda = master_panda.append(little_panda,ignore_index=True)
master_panda

In [ ]:
master_panda = master_panda.replace([np.inf, -np.inf], np.nan)

In [ ]:
master_panda = master_panda.dropna(axis=0)

In [ ]:
master_panda

In [ ]:
#master_panda.to_csv('/g/data/u46/users/ck9738/Datasets/ML_for_smad_green/training_datasets/indexis_test_2.csv')